In [1]:
import pandas as pd

products_to_identify = pd.read_csv("test_input.csv", chunksize=50)

In [2]:
from fuzzywuzzy import fuzz

/Users/danilodobras/.conda/envs/kollex_private_bi_analysis/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from sqlalchemy import create_engine

# Logging


pg_host =  "dwh-production.db.kollex.io"
pg_user =  "dwh"
pg_password =  "2QSw6bIRpa6aCKnn3P4uahJFId49640E2te0yt1vnPE91o1x17"

pg_database = "dwh"

pg_connect_string = f"postgresql://{pg_user}:{pg_password}@{pg_host}/{pg_database}"
pg_engine = create_engine(f"{pg_connect_string}", echo=False, pool_pre_ping=True, pool_recycle=36000)
chunk_size = 2000  # environ.get('CHUNK_SIZE')


connection = pg_engine.connect()

df_product = pd.read_sql("""
                                select concat( title , ' ' , brand , ' ' , amount_single_unit , 'x' , net_content ) as name_allskus
                                      , identifier

                                      , net_content::text         as "base_unit_content_allskus"
                                      , amount_single_unit::text          as "no_of_base_units_allskus"

                                    from prod_raw_layer.all_skus

                       """
, con=pg_engine)

In [ ]:
for chunk_num, chunk in enumerate(products_to_identify):
    start_index = chunk_num * chunk_size
    end_index = min(chunk_num * chunk_size + chunk_size, len(df_product))
    # .. do calculaton on chunk here ..
    df_product_joined = df_product.merge(chunk, how='cross')
    print("finished getting all SKUs from DB")

    df_product_joined['name_similarity'] = df_product_joined.apply(
        lambda x:
        fuzz.token_set_ratio(
            x['name_allskus'],
            x['name']
        ),
        axis=1)
    df_product_joined = df_product_joined[
        df_product_joined['name_similarity'] >= 80
        ]

    print("finished name_similarity and size is " + str(
        len(df_product_joined)))
    if len(df_product_joined.index) == 0:
        continue
    df_product_joined[
        'base_unit_content_similarity'] = df_product_joined.apply(
        lambda x:
        fuzz.token_set_ratio(
            str(x['base_unit_content_allskus']),
            str(x['base_unit_content'])
        ),
        axis=1)

    print("finished base_unit_content_similarity and size is " + str(
        len(df_product_joined)))
    if len(df_product_joined.index) == 0:
        continue
    df_product_joined['no_base_units_similarity'] = df_product_joined.apply(
        lambda x:
        fuzz.token_set_ratio(
            str(x['no_of_base_units_allskus']),
            str(x['no_of_base_units'])
        ),
        axis=1)
    print("finished no_base_units_similarity and size is " + str(
        len(df_product_joined)))

    if len(df_product_joined.index) == 0:
        continue
    df_product_joined = df_product_joined[
        (df_product_joined['base_unit_content_similarity'] >= 100) &
        (df_product_joined['no_base_units_similarity'] >= 100)]
    df_product_joined.drop_duplicates(subset=['name_allskus',
                                              'identifier',
                                              'gfgh_id',
                                              'name',
                                              'name_similarity'],
                                      inplace=True
                                      )
    df_product_joined.to_csv("result.csv", mode='a', header=False)
    

finished getting all SKUs from DB
finished name_similarity and size is 307
finished base_unit_content_similarity and size is 307
finished no_base_units_similarity and size is 307
finished getting all SKUs from DB
finished name_similarity and size is 723
finished base_unit_content_similarity and size is 723
finished no_base_units_similarity and size is 723
finished getting all SKUs from DB
finished name_similarity and size is 145
finished base_unit_content_similarity and size is 145
finished no_base_units_similarity and size is 145
finished getting all SKUs from DB
